In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from new_strategy import Asset, BetSizingMethod, get_bet_sizing
import nbimporter
from backtest import Backtest
from meta_strategy import MetaLabelingStrategy
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
import shap
import os
from datetime import datetime
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import brier_score_loss
from sklearn.calibration import calibration_curve
from scipy.stats import binom
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------------- MetaModelHandler ---------------------- #
class MetaModelHandler:
    def __init__(self):
        self.long_model = None
        self.short_model = None
        self.long_scaler = None
        self.short_scaler = None
        self.feature_cols = []

    def train(self, trades_df, long_feature_cols, short_feature_cols, asset_name, method_name):

        self.long_feature_cols = long_feature_cols
        self.short_feature_cols = short_feature_cols

        trades_df = trades_df.dropna(subset=['meta_label'])

        # Remove any 'set' column if it exists (we're not using train/val split anymore)
        if 'set' in trades_df.columns:
            trades_df = trades_df.drop('set', axis=1)

        # Prepare direction-specific data
        long_trades = trades_df[trades_df['direction'] == 'long'].dropna(subset=long_feature_cols)
        short_trades = trades_df[trades_df['direction'] == 'short'].dropna(subset=short_feature_cols)

        # =================== LONG MODEL ===================
        X_long = long_trades[long_feature_cols]
        y_long = long_trades["meta_label"]


        # Fit scaler on all long data
        X_long_scaled = X_long.values
        self.long_scaler = None

        self._sanity_check(X_long_scaled, y_long, "LONG")

        print("[Optuna] Tuning LONG model...")
        best_long_params = self.optimize_model_cv(X_long_scaled, y_long, n_trials=50)

        # Train final model with best parameters
        lgbm_long = LGBMClassifier(**best_long_params)

        # Use CalibratedClassifierCV with cross-validation (this is the key change)
    
        tscv = TimeSeriesSplit(n_splits=5)
        calibrated_long = CalibratedClassifierCV(
            lgbm_long, 
            method='isotonic',  # As recommended in the paper
            cv=tscv  # Use time series cross-validation instead of 'prefit'
        )

        # Fit the calibrated model (this will do CV internally)
        calibrated_long.fit(X_long_scaled, y_long)

        # Fit raw model for evaluation purposes
        lgbm_long.fit(X_long_scaled, y_long)

        self.long_model = calibrated_long
        self.raw_long_model = lgbm_long

        self.evaluate_calibration_cv(
            calibrated_model=calibrated_long,
            X=X_long_scaled,
            y=y_long,
            label="LONG"
        )

        X_long_df = pd.DataFrame(X_long_scaled, columns=self.long_feature_cols)

        self.plot_shap_values(
            model=lgbm_long,
            X=X_long,
            feature_names=self.long_feature_cols,
            title="Long",
            asset_name=asset_name,
            method_name=method_name
        )

        # =================== SHORT MODEL ===================
        X_short = short_trades[short_feature_cols]
        y_short = short_trades["meta_label"]

        # Fit scaler on all short data
        X_short_scaled = X_short.values
        self.short_scaler = None

        self._sanity_check(X_short_scaled, y_short, "SHORT")

        print("[Optuna] Tuning SHORT model...")
        best_short_params = self.optimize_model_cv(X_short_scaled, y_short, n_trials=50)

        # Train final model with best parameters
        lgbm_short = LGBMClassifier(**best_short_params)

        # Use CalibratedClassifierCV with cross-validation
        calibrated_short = CalibratedClassifierCV(
            lgbm_short, 
            method='isotonic',
            cv=tscv
        )

        # Fit the calibrated model
        calibrated_short.fit(X_short_scaled, y_short)

        # Fit raw model for evaluation purposes
        lgbm_short.fit(X_short_scaled, y_short)

        self.short_model = calibrated_short
        self.raw_short_model = lgbm_short

        self.evaluate_calibration_cv(
            calibrated_model=calibrated_short,
            X=X_short_scaled,
            y=y_short,
            label="SHORT"
        )

        X_short_df = pd.DataFrame(X_short_scaled, columns=self.short_feature_cols)

        self.plot_shap_values(
            model=lgbm_short,
            X=X_short,
            feature_names=self.short_feature_cols,
            title="Short",
            asset_name=asset_name,
            method_name=method_name
        )

    def optimize_model_cv(self, X, y, n_trials=150):
        """Optimize model hyperparameters using cross-validation"""
        def objective(trial):
            params = {
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
                "num_leaves": trial.suggest_int("num_leaves", 20, 64),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "random_state": 42,
                "n_jobs": -1,
                "verbosity": -1
            }

            model = LGBMClassifier(**params)
            tscv = TimeSeriesSplit(n_splits=5)
            score = cross_val_score(model, X, y, cv=tscv, scoring=make_scorer(f1_score)).mean()
            return score

        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=n_trials)

        print("✅ Best parameters:", study.best_params)
        print("📈 Best F1 score:", study.best_value)
        return study.best_params

    def evaluate_calibration(self, raw_model, calibrated_model, X, y, label):
        cal_probs = calibrated_model.predict_proba(X)[:, 1]
        print(f"\n📉 Calibration Evaluation — {label}")
        print(f"  → Brier score (calibrated): {brier_score_loss(y, cal_probs):.4f}")
        
        # ✅ Plot calibrated model calibration curve
        self.plot_calibration_with_ci(y_true=y, y_probs=cal_probs,
                                model_name="Calibrated Model", label=label)

        # Also evaluate raw model if provided
        if raw_model is not None:
            raw_probs = raw_model.predict_proba(X)[:, 1]
            print(f"  → Brier score (raw):        {brier_score_loss(y, raw_probs):.4f}")
            
            # ✅ Plot raw model calibration curve
            self.plot_calibration_with_ci(y_true=y, y_probs=raw_probs,
                                    model_name="Raw Model", label=label)
    
    def evaluate_calibration_cv(self, calibrated_model, X, y, label):
        print(f"\n📉 Calibration Evaluation (CV) — {label}")
        
        tscv = TimeSeriesSplit(n_splits=5)
        cv_probs_calibrated = []
        cv_probs_raw = []
        cv_true = []
        
        for train_idx, val_idx in tscv.split(X):
            X_train_fold, X_val_fold = X[train_idx], X[val_idx]
            y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
            
            # Handle different sklearn versions
            base_est = getattr(calibrated_model, "base_estimator", None)
            if base_est is None:
                base_est = getattr(calibrated_model, "estimator", None)
            base_model = LGBMClassifier(**base_est.get_params())
            base_model.fit(X_train_fold, y_train_fold)
            
            # Get RAW predictions on validation fold
            raw_probs = base_model.predict_proba(X_val_fold)[:, 1]
            cv_probs_raw.extend(raw_probs)
            
            # Calibrate on validation fold (paper's method)
            fold_calibrated = CalibratedClassifierCV(base_model, method='isotonic', cv='prefit')
            fold_calibrated.fit(X_val_fold, y_val_fold)
            
            # Evaluate on same validation fold (paper's method - optimistic but consistent)
            cal_probs = fold_calibrated.predict_proba(X_val_fold)[:, 1]
            cv_probs_calibrated.extend(cal_probs)
            
            cv_true.extend(y_val_fold)
        
        cv_probs_calibrated = np.array(cv_probs_calibrated)
        cv_probs_raw = np.array(cv_probs_raw)
        cv_true = np.array(cv_true)
        
        # Calculate Brier scores (will be optimistic for calibrated model)
        brier_calibrated = brier_score_loss(cv_true, cv_probs_calibrated)
        brier_raw = brier_score_loss(cv_true, cv_probs_raw)
        print(f"  → Brier score (raw):        {brier_raw:.4f}")
        print(f"  → Brier score (calibrated): {brier_calibrated:.4f}")
        print(f"  → Improvement:              {brier_raw - brier_calibrated:.4f}")
        
        # Plot comparison (calibrated will look very good)
        self.plot_calibration_comparison(
            y_true=cv_true, 
            y_probs_raw=cv_probs_raw,
            y_probs_calibrated=cv_probs_calibrated,
            label=label
        )

    
    def plot_calibration_comparison(self, y_true, y_probs_raw, y_probs_calibrated, label='LONG', n_bins=10):
        """Plot both raw and calibrated predictions on the same chart"""
        try:
            print(f"📊 Creating calibration comparison plot - {label}")
            
            y_true = np.array(y_true)
            y_probs_raw = np.array(y_probs_raw)
            y_probs_calibrated = np.array(y_probs_calibrated)
            
            if len(y_true) < 10:
                print(f"⚠️ Too few samples: {len(y_true)}")
                return
            
            actual_bins = min(10, len(y_true) // 10)
            if actual_bins < 2:
                actual_bins = 2
            
            # Get calibration curves for both
            frac_pos_raw, mean_pred_raw = calibration_curve(y_true, y_probs_raw, n_bins=actual_bins)
            frac_pos_cal, mean_pred_cal = calibration_curve(y_true, y_probs_calibrated, n_bins=actual_bins)
            
            # Calculate Brier scores
            brier_raw = np.mean((y_probs_raw - y_true) ** 2)
            brier_cal = np.mean((y_probs_calibrated - y_true) ** 2)
            
            # Create comparison plot
            fig, ax = plt.subplots(figsize=(10, 8))
            
            # Plot both curves
            ax.plot(mean_pred_raw, frac_pos_raw, 'o-', 
                    linewidth=2, markersize=8, label=f'Raw Model (Brier: {brier_raw:.4f})', color='red')
            ax.plot(mean_pred_cal, frac_pos_cal, 's-', 
                    linewidth=2, markersize=8, label=f'Calibrated Model (Brier: {brier_cal:.4f})', color='blue')
            
            # Perfect calibration reference line
            ax.plot([0, 1], [0, 1], '--', color='gray', label='Perfect Calibration')
            
            # Labels and formatting
            ax.set_xlabel('Mean Predicted Probability')
            ax.set_ylabel('Fraction of Positives') 
            ax.set_title(f'Calibration Comparison - {label}\nImprovement: {brier_raw - brier_cal:.4f}')
            ax.legend()
            ax.grid(True, alpha=0.3)
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            
            plt.tight_layout()
            plt.show()
            plt.close()
            
            print(f"✅ Calibration comparison plot created successfully")
            
        except Exception as e:
            print(f"❌ Calibration comparison plot failed: {e}")
            try:
                plt.close()
            except:
                pass

    def _sanity_check(self, X, y, label):
        """Sanity check for data quality"""
        print(f"\n📊 Sanity Check for {label} dataset")
        print("  → Shape:", X.shape)
        print("  → NaNs in X:", np.isnan(X).sum())
        print("  → All-zero columns:", (X == 0).all(axis=0).sum())
        print("  → y balance:", np.bincount(y.astype(int)) if len(np.unique(y)) == 2 else y.value_counts())
                    
    def plot_shap_values(self, model, X, feature_names, title, asset_name, method_name):
            
        plt.close()
        plt.style.use('default')

        print(f"[SHAP] Generating plot for: {title}")
            
        # If using CalibratedClassifierCV, get the base estimator
        if hasattr(model, "base_estimator_"):
            model = model.base_estimator_

        # Create SHAP explainer
        explainer = shap.TreeExplainer(model)
        shap_values = explainer(X)

        # Plot SHAP bar chart
        shap.plots.bar(shap_values, max_display=len(feature_names), show=False)

        fig = plt.gcf()
        fig.suptitle(
            f"SHAP Feature Importance — {title.capitalize()} — {asset_name.upper()} — {method_name.upper()}",
            fontsize=14
        )
        plt.tight_layout(rect=[0, 0, 1, 0.95])

        # Output directory
        output_dir = "results_metalabel/shap"
        os.makedirs(output_dir, exist_ok=True)

        # Construct filename
        safe_title = title.lower().replace(" ", "_")
        filename = f"shap_{asset_name.lower()}_{method_name.lower()}_{safe_title}.png"
        full_path = os.path.join(output_dir, filename)

        # Save image
        plt.savefig(full_path, dpi=300)
        plt.close()
        print(f"[SHAP] Saved to {full_path}")

        # Save mean absolute SHAP values summary
        mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
        shap_summary_df = pd.DataFrame({
            'feature': feature_names,
            'mean_abs_shap_value': mean_abs_shap
        }).sort_values(by='mean_abs_shap_value', ascending=False)

        summary_filename = f"shap_summary_{asset_name.lower()}_{method_name.lower()}_{safe_title}.csv"
        summary_path = os.path.join(output_dir, summary_filename)
        shap_summary_df.to_csv(summary_path, index=False)
        print(f"[SHAP] Summary CSV saved to {summary_path}")    

    def is_trade_approved(self, features: dict, direction: str, threshold: float = 0.3) -> bool:
        if direction == 'long':
            feature_list = self.long_feature_cols
            model = self.long_model
        else:
            feature_list = self.short_feature_cols
            model = self.short_model
            

        cleaned = {}
        for k in feature_list:
            val = features.get(k, 0)
            if pd.isna(val) or val in [np.inf, -np.inf]:
                cleaned[k] = 0
            else:
                cleaned[k] = val

        df = pd.DataFrame([cleaned])[feature_list]
        X = df.values  # Use raw values, no scaling
        prob = model.predict_proba(X)[0, 1]

        print(f"[MetaModel] Direction: {direction}, Prob: {prob:.3f}, Threshold: {threshold}, Approved: {prob >= threshold}")
        return prob >= threshold

    def plot_calibration_with_ci(self, y_true, y_probs, model_name='Calibrated Model', label='LONG', n_bins=10):
        """Simple replacement - no confidence intervals, just basic calibration plot"""
        try:
            print(f"📊 Creating simple calibration plot for {model_name} - {label}")
            
            # Convert to numpy arrays
            y_true = np.array(y_true)
            y_probs = np.array(y_probs)
            
            # Basic checks
            if len(y_true) < 10:
                print(f"⚠️ Too few samples: {len(y_true)}")
                return
            
            # Use fewer bins for reliability
            actual_bins = min(5, len(y_true) // 10)
            if actual_bins < 2:
                actual_bins = 2
            
            # Get calibration curve
            fraction_of_positives, mean_predicted_value = calibration_curve(
                y_true, y_probs, n_bins=actual_bins
            )
            
            # Calculate Brier score
            brier = np.mean((y_probs - y_true) ** 2)
            
            # Create simple plot
            fig, ax = plt.subplots(figsize=(8, 8))
            
            # Plot calibration line
            ax.plot(mean_predicted_value, fraction_of_positives, 'o-', 
                    linewidth=2, markersize=8, label=f'{model_name}')
            
            # Perfect calibration reference line
            ax.plot([0, 1], [0, 1], '--', color='gray', label='Perfect Calibration')
            
            # Labels and formatting
            ax.set_xlabel('Mean Predicted Probability')
            ax.set_ylabel('Fraction of Positives') 
            ax.set_title(f'Calibration Plot - {label} - {model_name}\nBrier Score: {brier:.4f}')
            ax.legend()
            ax.grid(True, alpha=0.3)
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            
            plt.tight_layout()
            plt.show()
            plt.close()
            
            print(f"✅ Calibration plot created successfully")
            
        except Exception as e:
            print(f"❌ Calibration plot failed: {e}")
            try:
                plt.close()
            except:
                pass    


"""def train_meta_model(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    long_feature_cols: list,
    short_feature_cols: list,
    asset,
    method
) -> MetaModelHandler:
    # Shift rolling metrics to avoid lookahead bias
    rolling_cols = [
        'rolling_f1', 'rolling_accuracy', 'rolling_precision', 'rolling_recall',
        'n_total_seen', 'n_window_obs'
    ]
    for col in rolling_cols:
        if col in train_df.columns:
            train_df[col] = train_df.groupby('session')[col].shift(1)
    
    combined_df = pd.concat([train_df, val_df])
    split_index = len(train_df)  

    meta_model = MetaModelHandler()
    meta_model.train(train_df, long_feature_cols, short_feature_cols,asset.value, method.value,split_index)
    return meta_model"""

def optimize_model(X, y, n_trials=150):
    def objective(trial):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "num_leaves": trial.suggest_int("num_leaves", 20, 64),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "random_state": 42,
            "n_jobs": -1,
            "verbosity": -1
        }

        model = LGBMClassifier(**params)

        tscv = TimeSeriesSplit(n_splits=5)
        score = cross_val_score(model, X, y, cv=tscv, scoring=make_scorer(f1_score)).mean()
        return score

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials)

    print("✅ Best parameters:", study.best_params)
    print("📈 Best F1 score:", study.best_value)
    return study.best_params
